In [12]:
import pandas as pd
import gzip
import json
import os
import shutil
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect
import calendar

load_dotenv('../.env')

True

Datasets directory

In [2]:
DIR = '../data/external/'

loading datasets

In [5]:
books_path = os.path.join(DIR, 'goodreads_books_history_biography.json.gz')
tmpfile = os.path.join(DIR, 'books.json')

# Create temporary file
with open(tmpfile, "wb") as tmp:
    shutil.copyfileobj(gzip.open(books_path), tmp)
    
# dataframe from temporary file
books = pd.read_json(tmpfile, lines=True)

# delete temporary file
os.remove(tmpfile)

In [3]:
review_path = os.path.join(DIR, 'goodreads_reviews_history_biography.json.gz')
tmpfile = os.path.join(DIR, 'reviews.json')

# Create temporary file
with open(tmpfile, "wb") as tmp:
    shutil.copyfileobj(gzip.open(review_path), tmp)
    
# dataframe from temporary file
reviews = pd.read_json(tmpfile, lines=True)

# delete temporary file
os.remove(tmpfile)

In [38]:
db_url = f"postgresql://{os.getenv('DB_USERNAME')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(db_url)

Get columns in the tables in our database

In [39]:
inspector = inspect(engine)
db_tables = ['books', 'reviews']
b_cols, r_cols = [],[]
tables_cols = [b_cols, r_cols]
for i, table in enumerate(db_tables):
    columns = inspector.get_columns(table)
    print(f' == ({table}) ==')
    for col in columns:
        print(f"Column Name: {col['name']}, Type: {col['type']}")
        tables_cols[i].append(col['name'])

 == (books) ==
Column Name: book_id, Type: INTEGER
Column Name: title, Type: VARCHAR
Column Name: ratings_count, Type: INTEGER
Column Name: publication_year, Type: INTEGER
Column Name: publication_month, Type: INTEGER
Column Name: publication_day, Type: INTEGER
Column Name: average_rating, Type: INTEGER
Column Name: description, Type: VARCHAR
Column Name: authors, Type: JSONB
 == (reviews) ==
Column Name: book_id, Type: INTEGER
Column Name: review_text, Type: VARCHAR
Column Name: n_votes, Type: INTEGER
Column Name: n_comments, Type: INTEGER
Column Name: rating, Type: INTEGER


populating tables in database

In [46]:
set(b_cols).issubset(set(books.columns))

True

In [49]:
set(r_cols).issubset(set(reviews.columns))

True

convert `authors` column to json

In [70]:
books['authors'] = books['authors'].apply(lambda x: json.dumps(x))

In [71]:
books[b_cols].to_sql('books', engine, if_exists='replace', index=False)
reviews[r_cols].to_sql('reviews', engine, if_exists='replace', index=False)

193